In [5]:
import pandas as pd
# from Bio import SeqIO
from sklearn.model_selection import train_test_split
from IO import make_fasta
import re


In [6]:
path_file = 'data/viral_fp_curated.csv'
curated_dataset = pd.read_csv(path_file)
curated_dataset

,idProtein,Name,Class,Activation,Name_Fusogenic_Unit,Location_Fusogenic,Sequence_fusogenic,UniProtID,NcbiID,idTaxonomy,...,idTaxonomy.1,CommonName,Family,Genre,Species,SubSpecies,NcbiTax,FP corrected DL,Notes DL,seq_vfp
0,2,Envelope glycoprotein,I,binding to receptor,Envelope protein p15E,471-650,EPVSLTLALLLGGLTMGGIAAGVGTGTTALVATQQFQQLQAAMHDD...,P03386,NaN,2,...,2,AKV murine leukemia virus (AKR (endogenous) mu...,Retroviridae,Gammaretrovirus,AKR murine leukemia virus,NaN,11791,NaN,NaN,VSLTLALLLGGLTMGGIAAGV
1,4,Envelope glycoprotein gp95,I,binding to receptor,gp37,402-606,STSHLDDTCSDEVQLWGPTARIFASILAPGVAAAQALREIERLACW...,P03397,NaN,4,...,4,Avian leukosis virus RSA (RSV-SRA) (Rous sarco...,Retroviridae,Alpharetrovirus,Rous sarcoma virus,strain Schmidt-Ruppin A,269446,NaN,NaN,GPTARIFASILAPGVAAAQAL
2,5,Fusion glycoprotein F0,I,binding to receptor,F1,103-537,FVLGAIALGVATAAAVTAGVAIAKTIRLEGEVAAIKGALRKTNEAV...,Q2Y2M3,NaN,5,...,5,Avian metapneumovirus (isolate Canada goose/Mi...,Pneumoviridae,Metapneumovirus,Avian metapneumovirus,15a,652954,NaN,NaN,FVLGAIALGVATAAAVTAGVAI
3,6,Envelope glycoprotein,I,binding to receptor,gp22,393-582,AVQFIPLLVGLGITGATLAGGTGLGVSVHTYHKLSNQLIEDVQALS...,P03399,NaN,6,...,6,Avian reticuloendotheliosis virus,Retroviridae,Gammaretrovirus,Reticuloendotheliosis virus,NaN,11636,NaN,NaN,FIPLLVGLGITGATLAGGTGL
4,7,Envelope glycoprotein,I,binding to receptor,gp37,79-257,SVSHLDDTCSDEVQLWGPTARIFASILAPGVAAAQALREIERLACW...,P33498,NaN,7,...,7,Avian retrovirus RPL30,Retroviridae,Alpharetrovirus,Avian leukosis virus,RPL30,31671,NaN,NaN,GPTARIFASILAPGVAAAQAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,798,Genome polyprotein,NaN,binding to receptor,Envelop protein E,286 - 778,AHCIGVADRDFIEGVHGGTWVSATLEQDKCVTVMAPDKPSLDISLE...,Q9YRV3,NaN,798,...,798,Yellow fever virus,Flaviviridae,Flavivirus,Yellow fever virus,Trinidad/79A/1979,407137,NaN,NaN,DRGWGNGCGLFGKG
399,799,Genome polyprotein,NaN,binding to receptor,Envelop protein E,286 - 778,AHCIGITDRDFIEGVHGGTWVSASLEQDKCVTVMAPDKPSLDISLQ...,Q1X880,NaN,799,...,799,Yellow fever virus,Flaviviridae,Flavivirus,Yellow fever virus,isolate Uganda/A7094A4/1948,407139,NaN,NaN,DRGWGNGCGLFGKG
400,800,Genome polyprotein,NaN,binding to receptor,Envelop protein E,286 - 778,AHCIGITDRDFIEGVHGGTWVSATLEQGKCVTVMAPDKPSLDISLQ...,Q074N0,NaN,800,...,800,Yellow fever virus,Flaviviridae,Flavivirus,Yellow fever virus,isolate Ethiopia/Couma/1961,407141,NaN,NaN,DRGWGNGCGLFGKG
401,801,Genome polyprotein,NaN,binding to receptor,Envelop protein E,268 - 778,AHCIGITDRDFIEGVHGGTWVSATLEQDKCVTVMAPDKPSLDISLE...,Q89277,NaN,801,...,801,Yellow fever virus,Flaviviridae,Flavivirus,Yellow fever virus,strain French neurotropic vaccine,407135,NaN,NaN,DRGWGNGCGLFGKG


In [3]:
#   cd-hit -i nr -o nr100 -c 1.00 -n 5 -M 16000 –d 0 -T 8
#   cd-hit -i db -o db90 -c 0.9 -n 5 -M 16000 –d 0 -T 8
#   where
#   db is the filename of input,
#   db90 is output, 
#   -c 1.0, means 100% identity, is the clustering threshold
#   -c 0.9, means 90% identity, is the clustering threshold
#   -n 5 is the word size
#   -d 0 use sequence name in fasta header till the first white space
#   -M 16000, to use 16GB RAM
#   -T 8, to use 8 threads
# Choose of word size:
# -n 5 for thresholds 0.7 ~ 1.0
# -n 4 for thresholds 0.6 ~ 0.7
# -n 3 for thresholds 0.5 ~ 0.6
# -n 2 for thresholds 0.4 ~ 0.5


# protein dataset in fasta format 
# output are two files: a fasta file of representative sequences and a text file of list of clusters.


# make fasta
from IO import make_fasta, read_fasta, read_cluster_file, fasta_to_dict

make_fasta(df = curated_dataset , filename_fasta = 'fasta_curated.fasta' ,seq_column = 'seq_vfp')

# create the dict with names and sequences
dict_or = fasta_to_dict(fasta_or = 'fasta_curated.fasta')

# functin to get the clusters with just only one sequence
def get_keys_with_only_one_sequence(clusters):
    cluster_single = dict()
    for key in clusters.keys():
        if len(clusters[key]) == 1: 
#             print(clusters[key])
            cluster_single[key] = clusters[key]
    print(len(cluster_single.keys()))
    return cluster_single


# clustering 90 % similarity 
!cd-hit -i fasta_curated.fasta -o results_cdhit90 -c 0.9 -n 5 -M 16000 -d 0 -T 8

seq90_fasta = read_fasta('results_cdhit90', th=90)
seq90_cluster = 'results_cdhit90.clstr'
clusters90 = read_cluster_file(cl_file=seq90_cluster, dict_or=dict_or)
cluster90_single = get_keys_with_only_one_sequence(clusters90)


# clustering 80 % similarity 
!cd-hit -i fasta_curated.fasta -o results_cdhit80 -c 0.8 -n 5 -M 16000 -d 0 -T 8

seq80_fasta = read_fasta('results_cdhit80', th=80)
seq80_cluster = 'results_cdhit80.clstr'
clusters80 = read_cluster_file(cl_file=seq80_cluster, dict_or=dict_or)
cluster80_single = get_keys_with_only_one_sequence(clusters80)


# clustering 70 % similarity 
!cd-hit -i fasta_curated.fasta -o results_cdhit70 -c 0.7 -n 5 -M 16000 -d 0 -T 8

seq70_fasta = read_fasta('results_cdhit70', th=70)
seq70_cluster = 'results_cdhit70.clstr'
clusters70 = read_cluster_file(cl_file=seq70_cluster, dict_or=dict_or)
cluster70_single = get_keys_with_only_one_sequence(clusters70)


# clustering 60 % similarity    word len is 3
!cd-hit -i fasta_curated.fasta -o results_cdhit60 -c 0.6 -n 3 -M 16000 -d 0 -T 8

seq60_fasta = read_fasta('results_cdhit60', th=60)
seq60_cluster = 'results_cdhit60.clstr'
clusters60 = read_cluster_file(cl_file=seq60_cluster, dict_or=dict_or)
cluster60_single = get_keys_with_only_one_sequence(clusters60)




# clustering 50 % similarity word len is 3
!cd-hit -i fasta_curated.fasta -o results_cdhit50 -c 0.5 -n 3 -M 16000 -d 0 -T 8

seq50_fasta = read_fasta('results_cdhit50', th=50)
seq50_cluster = 'results_cdhit50.clstr'
clusters50 = read_cluster_file(cl_file=seq50_cluster, dict_or=dict_or)
cluster50_single = get_keys_with_only_one_sequence(clusters50)

Program: CD-HIT, V4.8.1 (+OpenMP), Apr 07 2021, 10:57:21
Command: cd-hit -i fasta_curated.fasta -o results_cdhit90 -c
         0.9 -n 5 -M 16000 -d 0 -T 8

Started: Mon Dec 26 16:27:18 2022
                            Output                              
----------------------------------------------------------------
Discarding invalid sequence or sequence without identifier and description!

>nan_G
[FRWYGPKY CGYATVT]
total seq: 402
longest and shortest : 47 and 13
Total letters: 9063
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 8 X 10M = 84M
Table           : 2 X 65M = 130M
Miscellaneous   : 0M
Total           : 214M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 1973149542

# comparing sequences from          0  to        402
---------- new table with      120 representatives

      402  finished        120  clusters

Approximated maximum memory consu

In [4]:
# create a table 
import tabulate


### similarity thres  number of clusters     number of custers with seqs alone

data = [
    ["90%",len(clusters90.keys()),len(cluster90_single.keys())],
    ["80%",len(clusters80.keys()),len(cluster80_single.keys())],
    ["70%",len(clusters70.keys()),len(cluster70_single.keys())],
    ["60%",len(clusters60.keys()),len(cluster60_single.keys())],
    ["50%",len(clusters50.keys()),len(cluster50_single.keys())]]
table = tabulate.tabulate(data,tablefmt='html') # , 
table

90%,119,52
80%,90,29
70%,67,15
60%,49,12
50%,39,8


There are 52 sequences with only one sequence with threshold 90%. 
There are 29 sequences with only one sequence with threshold 80%.
There are 15 sequences with only one sequence with threshold 70%.
There are 12 sequences with only one sequence with threshold 60%.
There are 8 sequences with only one sequence with threshold 50%.



The idea is taking sequences with low similarity but garantee that it will not have a similar protein in the train dataset. This is important as we will consider all the protein sequences even with high similarity. So, if it is  sequence with only one hit in 70 similarity we will guarantee that will not be any protein similar in the train dataset even if we use a similarity of 90 %. 


The threshold of 50% and 60 &similarity is to conservative and will lead to lost of important sequences to the train dataset, take 6 sequences from this clusters will take to much info. 90% similarity cover too much sequences and may lead to overestimate scores. 

We will therefore take the final test sequences to evaluate the model in full lenght sequences using the 80 % threshold. 


To make the test dataset we will use the bipartide peptide ( that is not take into account in the CDHIT), and 8 more sequences from the 80 % similarity. More sequences may be used, but has the dataset is very small we consider that this would be enough. 
The machine learning models will be evaluated while training with different test sets. This partition intends to qualitatively evaluate the model on full lenght sequences. 


To choose from the similarity we will choose from clusters with only one sequence (to guarantee that are not similar proteins in train set) and we will take into consideration the distribution of classes. We will just consider sequences with fusogenic sequence, Class and uniprot ID. 


In [5]:
values_test = [str(seq[0]) for seq in cluster80_single.values()]
cluster_df = curated_dataset.loc[curated_dataset['seq_vfp'].isin(values_test)]
cluster_df


,idProtein,Name,Class,Activation,Name_Fusogenic_Unit,Location_Fusogenic,Sequence_fusogenic,UniProtID,NcbiID,idTaxonomy,...,idTaxonomy.1,CommonName,Family,Genre,Species,SubSpecies,NcbiTax,FP corrected DL,Notes DL,seq_vfp
6,9,Envelope glycoprotein,I,binding to receptor,Glycoprotein p20E,377-546,AIQFIPLLAGLGITAAFTTGATGLGVSVTQYTKLSNQLISDVQILS...,P10269,NaN,9,...,9,Baboon endogenous virus (strain M7),Retroviridae,Gammaretrovirus,Baboon endogenous virus,strain M7,11764,NaN,NaN,FIPLLAGLGITAAFTTGATGL
15,19,Envelope glycoprotein,I,binding to receptor,gp40,556-904,AVGLAIFLLVLAIMAITSSLVAATTLVNQHTTAKVVERVVQNVSYI...,P19557,NaN,19,...,19,Bovine immunodeficiency virus (strain R29) (BI...,Retroviridae,Lentivirus,Bovine immunodeficiency virus,strain R29,417296,NaN,NaN,AVGLAIFLLVLAIMAITSSLV
36,54,Glycoprotein precursor,II,low pH,NaN,NaN,NaN,NaN,NP_950235.1,54,...,54,Crimean-Congo hemorrhagic fever virus,Nairoviridae,Orthonairovirus,NaN,NaN,1980519,NaN,NaN,WRCNPTWCWGVGTGCTCCGLDVK
68,97,Envelope glycoprotein gp130,I,binding to receptor,gp48,564-982,SVSTENLRRIQEAGLGLANAITTVAKISDLNDQKLAKGVHLLRDHV...,O56861,NaN,97,...,97,Feline foamy virus (FFV) (Feline syncytial virus),Retroviridae,Felispumavirus,Feline foamy virus,NaN,53182,NaN,NaN,NLRRIQEAGLGLANAITTVAKIS
76,105,Envelope glycoprotein,I,binding to receptor,Envelope protein p15E,466-644,DPISLTVALMLGGITVGGMARNRNRDCGLLETAQFRQLQMAMHTDI...,P21443,NaN,105,...,105,Feline leukemia virus (isolate CFE-6),Retroviridae,Gammaretrovirus,Feline leukemia virus,clone CFE-6,11922,NaN,NaN,PISLTVALMLGGITVGGMARN
85,115,Polyprotein,NaN,binding to receptors,E2,207-642,SEFWRWTEQLASNYWILEYLWKVPFEFWRGVMSLTPLLVWVAALLL...,NaN,AF121950.1,115,...,115,GB virus C/Hepatitis G virus,Flaviviridae,Pegivirus,Pegivirus C,C,54290,NaN,NaN,GTEVSEALGGAGLTGGFY
91,126,Large envelope protein,NaN,NaN,S protein,164-389,MENITSGFLGPLLVLQAGFFLLTRILTIPQSLDSWWTSLNFLGGTT...,P03138,X02496,126,...,126,Hepatitis B virus,Hepadnaviridae,Orthohepadnavirus,Hepatitis B virus,ayw/France/Tiollais/1979,490133,NaN,NaN,MENITSGFLGPLLVLQAGFFLLTR
107,143,Genome polyprotein,II,NaN,Envelope glycoprotein E1,192-383,VNYRNVSGIYHVTNDCPNSSIVYEADHHIMHLPGCVPCVREGNQSR...,O39929,NaN,143,...,143,Hepatitis C virus,Flaviviridae,Hepacivirus,Hepacivirus C,ED43,356418,NaN,NaN,MVGAATVCSGLYIGDLCGGLFLVGQMFSFRPR
110,146,Genome polyprotein,II,NaN,Envelope glycoprotein E1,192-384,LEYRNASGLYLLTNDCSNRSIVYEADDVILHLPGCVPCVETDNNNT...,Q81487,NaN,146,...,146,Hepatitis C virus,Flaviviridae,Hepacivirus,Hepacivirus C,isolate Tr Kj,357355,NaN,NaN,LVAPPTLCSALYVEDAFGAVSLVGQAFTFRPR
113,149,Genome polyprotein,II,NaN,Envelope glycoprotein E1,192-383,VEVRNTSSSYMATNDCSNSSIVWQLEGAVLHTPGCVPCEKTGNKSR...,Q68749,NaN,149,...,149,Hepatitis C virus,Flaviviridae,Hepacivirus,Hepacivirus C,isolate BEBE1,356413,NaN,NaN,IVMSATLCSALYVGDVCGALMIAAQVVVVSPQ


In [6]:
cluster_df_clean = cluster_df.dropna(subset=['Sequence_fusogenic', 'UniProtID', 'Class'])
cluster_df_clean

,idProtein,Name,Class,Activation,Name_Fusogenic_Unit,Location_Fusogenic,Sequence_fusogenic,UniProtID,NcbiID,idTaxonomy,...,idTaxonomy.1,CommonName,Family,Genre,Species,SubSpecies,NcbiTax,FP corrected DL,Notes DL,seq_vfp
6,9,Envelope glycoprotein,I,binding to receptor,Glycoprotein p20E,377-546,AIQFIPLLAGLGITAAFTTGATGLGVSVTQYTKLSNQLISDVQILS...,P10269,NaN,9,...,9,Baboon endogenous virus (strain M7),Retroviridae,Gammaretrovirus,Baboon endogenous virus,strain M7,11764,NaN,NaN,FIPLLAGLGITAAFTTGATGL
15,19,Envelope glycoprotein,I,binding to receptor,gp40,556-904,AVGLAIFLLVLAIMAITSSLVAATTLVNQHTTAKVVERVVQNVSYI...,P19557,NaN,19,...,19,Bovine immunodeficiency virus (strain R29) (BI...,Retroviridae,Lentivirus,Bovine immunodeficiency virus,strain R29,417296,NaN,NaN,AVGLAIFLLVLAIMAITSSLV
68,97,Envelope glycoprotein gp130,I,binding to receptor,gp48,564-982,SVSTENLRRIQEAGLGLANAITTVAKISDLNDQKLAKGVHLLRDHV...,O56861,NaN,97,...,97,Feline foamy virus (FFV) (Feline syncytial virus),Retroviridae,Felispumavirus,Feline foamy virus,NaN,53182,NaN,NaN,NLRRIQEAGLGLANAITTVAKIS
76,105,Envelope glycoprotein,I,binding to receptor,Envelope protein p15E,466-644,DPISLTVALMLGGITVGGMARNRNRDCGLLETAQFRQLQMAMHTDI...,P21443,NaN,105,...,105,Feline leukemia virus (isolate CFE-6),Retroviridae,Gammaretrovirus,Feline leukemia virus,clone CFE-6,11922,NaN,NaN,PISLTVALMLGGITVGGMARN
107,143,Genome polyprotein,II,NaN,Envelope glycoprotein E1,192-383,VNYRNVSGIYHVTNDCPNSSIVYEADHHIMHLPGCVPCVREGNQSR...,O39929,NaN,143,...,143,Hepatitis C virus,Flaviviridae,Hepacivirus,Hepacivirus C,ED43,356418,NaN,NaN,MVGAATVCSGLYIGDLCGGLFLVGQMFSFRPR
110,146,Genome polyprotein,II,NaN,Envelope glycoprotein E1,192-384,LEYRNASGLYLLTNDCSNRSIVYEADDVILHLPGCVPCVETDNNNT...,Q81487,NaN,146,...,146,Hepatitis C virus,Flaviviridae,Hepacivirus,Hepacivirus C,isolate Tr Kj,357355,NaN,NaN,LVAPPTLCSALYVEDAFGAVSLVGQAFTFRPR
113,149,Genome polyprotein,II,NaN,Envelope glycoprotein E1,192-383,VEVRNTSSSYMATNDCSNSSIVWQLEGAVLHTPGCVPCEKTGNKSR...,Q68749,NaN,149,...,149,Hepatitis C virus,Flaviviridae,Hepacivirus,Hepacivirus C,isolate BEBE1,356413,NaN,NaN,IVMSATLCSALYVGDVCGALMIAAQVVVVSPQ
117,153,Genome polyprotein,II,NaN,Envelope glycoprotein E1,192-383,VEVKNTSQIYMATNDCSNNSITWQLEGAVLHVPGCVPCESTGNISR...,Q9QAX1,NaN,153,...,153,Hepatitis C virus,Flaviviridae,Hepacivirus,Hepacivirus C,isolate VAT96,356414,NaN,NaN,IVVSATFCSALYIGDVCGAIMIAAQATIISPQ
122,158,Genome polyprotein,II,NaN,Envelope glycoprotein E1,192-383,LEYRNASGLYTVTNDCSNGSIVYEAGDVILHLPGCIPCVRLNNASK...,Q68801,NaN,158,...,158,Hepatitis C virus,Flaviviridae,Hepacivirus,Hepacivirus C,isolate JK049,356417,NaN,NaN,MVGAATLCSALYVGDLCGALFLVGQGFSWRHR
135,192,Spike glycoprotein S,I,interacting with receptor,Corona S2,759-1353,AITTGYRFTNFEPFTVNSVNDSLEPVGGLYEIQIPSEFTIGNMVEF...,P36334,NaN,192,...,192,Human Coronavirus,Coronaviridae,Betacoronavirus,Betacoronavirus 1,OC43,31631,NaN,NaN,LAATSASLFPPWTAAAGVPFY


In [11]:
from sklearn.model_selection import train_test_split
cluster_df_clean
X_train, X_test, y_train, y_test = train_test_split(cluster_df_clean, 
                                                    cluster_df_clean['Class'], 
                                                    test_size=0.33, 
                                                    stratify = cluster_df_clean['Class'], 
                                                    random_state=42)

print(X_test)

     idProtein                                            Name Class  \
135        192                            Spike glycoprotein S     I   
205        265  Envelope glycoprotein gp160 (Env polyprotein)      I   
76         105                           Envelope glycoprotein     I   
308        618                          fusion glycoprotein F0     I   
283        574         Pre-glycoprotein polyprotein GP complex     I   
122        158                              Genome polyprotein    II   
110        146                              Genome polyprotein    II   
165        225  Envelope glycoprotein gp160 (Env polyprotein)      I   

                    Activation       Name_Fusogenic_Unit Location_Fusogenic  \
135  interacting with receptor                 Corona S2           759-1353   
205        binding to receptor      gp41 - by similarity            488-832   
76         binding to receptor     Envelope protein p15E            466-644   
308        binding to receptor     

To this 8 sequences was added the bipartite peptide (IdProtein 779) and the fusion peptide of Srascov2 (that has still not an established fusion peptide) from the original curated database,  making a test dataset of 10 sequences. 
